Group Name: AG xx.

Student Name (Student ID):

1. xxxx xxxxx (xxxxxxx)

2. xxxx xxxxx (xxxxxxx)

3. xxxx xxxxx (xxxxxxx)

4. xxxx xxxxx (xxxxxxx)

# Question 1

Consider the maze shown below. The Maze has 16 rows and 24 columns The objective is to find a shortest path from cell $S$ to cell $G$.


![Maze](Maze.jpg)


The agent can take four actions in each cell: 'RIGHT', 'DOWN', 'UP', 'LEFT'.  

Each cell is represented as $(x,y)$, where $x$ indicates row number and $y$ indicates column number. Action 'UP' takes the agent from cell $(x,y)$ to $(x+1,y)$. Action 'DOWN' takes the agent from cell $(x,y)$ to $(x-1,y)$. Action 'RIGHT' takes the agent from cell $(x,y)$ to $(x,y+1)$. Action 'LEFT' takes the agent from cell $(x,y)$ to $(x,y-1)$. The triplet $(s,a,s')$  indicates that taking action $a$ at state $s$ leads to state $s'$. Actions 'LEFT' or 'RIGHT' cost 10 units for all $(s,a,s')$. Actions 'UP' or 'DOWN' cost 1 unit for all  $(s,a,s')$.  The agent cannot move into cells that are shaded. Assume that the agent knows the boundaries of the maze and has full observability. Consequently, at the bottom (row 0) and top (row 15), the agent will not take actions 'DOWN' and 'UP', respectively; at left (column 0) and right (column 23) columns, the agent will not take 'LEFT' and 'RIGHT' actions, respectively. Similalry, the agent will not take actions that lead to shaded region in the maze.

## **Q1.a: Class Maze(Problem)** [5 Marks]

Write a Maze class to create a model for this problem. You should not use an explicit state space model. The modelling should inherit the abstract class 'Problem' (given below). With the problem formulation, find the shortest path from S to G cell. Propose and implement multiple heuristics (at least two heuristics) for informed search algorithms.

## **Q1.b: Analysis of the Algorithms** [5 Marks]

1. Solve the above Maze problem using the following algorithms

    a. Breadth-First Search

    b. Depth-First Search with Cycle-Check

    c. Iterative-Deepening Search with Cycle-Check

    d. Uniform-Cost Search

    e. A* Search 

    f. Greedy Best-first Search

2. Identify the number of number of expanded nodes, maximum frontier size, and path-cost for the above algorithms.  Summarize the statistics in the form of a table.
 
3. Compare the performance of informed search algorithms with proposed heuristics. Identify the best performing heuristic and explain.
 

Note 1: You must follow the problem formulation discussed in the class. A abstract class for Problem amd Node definition is presented below. The search tree generation should follow the template discussed in the class (i.e., Node class, expand methods, etc.). 

Note 2: The code should be written in a single jupyter notebook file.

In [1]:
class Problem:
    """The abstract class for a formal problem. A new domain subclasses this,
    overriding `actions` and `results`, and perhaps other methods.
    The default heuristic is 0 and the default action cost is 1 for all states.
    When you create an instance of a subclass, specify `initial`, and `goal` states 
    (or give an `is_goal` method) and perhaps other keyword args for the subclass."""

    def __init__(self, initial=None, goal=None, **kwds): 
        self.__dict__.update(initial=initial, goal=goal, **kwds) 
        
    def actions(self, state):        raise NotImplementedError
    def result(self, state, action): raise NotImplementedError
    def is_goal(self, state):        return state == self.goal
    def action_cost(self, s, a, s1): return 1
    def h(self, node):               return 0
    
    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)

In [2]:
# Use the following Node class to generate search tree
import math
class Node:
    "A Node in a search tree."
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)

    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.path_cost < other.path_cost 


In [3]:
failure = Node('failure', path_cost=math.inf) # Indicates an algorithm couldn't find a solution.
cutoff  = Node('cutoff',  path_cost=math.inf) # Indicates iterative deepening search was cut off.
  
def expand(problem, node):
    "Expand a node, generating the children nodes."
    s = node.state
    for action in problem.actions(s):
        s1 = problem.result(s, action)
        cost = node.path_cost + problem.action_cost(s, action, s1)
        yield Node(s1, node, action, cost)
        

def path_actions(node):
    "The sequence of actions to get to this node."
    if node.parent is None:
        return []  
    return path_actions(node.parent) + [node.action]


def path_states(node):
    "The sequence of states to get to this node."
    if node in (cutoff, failure, None): 
        return []
    return path_states(node.parent) + [node.state]

In [4]:
from collections import deque
import heapq


FIFOQueue = deque

LIFOQueue = list

class PriorityQueue:
    """A queue in which the item with minimum f(item) is always popped first."""

    def __init__(self, items=(), key=lambda x: x): 
        self.key = key
        self.items = [] # a heap of (score, item) pairs
        for item in items:
            self.add(item)
         
    def add(self, item):
        """Add item to the queuez."""
        pair = (self.key(item), item)
        heapq.heappush(self.items, pair)

    def pop(self):
        """Pop and return the item with min f(item) value."""
        return heapq.heappop(self.items)[1]
    
    def top(self): return self.items[0][1]

    def __len__(self): return len(self.items)

## **Q1.a: Class Maze(Problem)** [5 Marks]

Write a Maze class to create a model for this problem. You should not use an explicit state space model. The modelling should inherit the abstract class 'Problem' (given below). With the problem formulation, find the shortest path from S to G cell. Propose and implement multiple heuristics (at least two heuristics) for informed search algorithms.

In [5]:
MAZE_ROWS = 16
MAZE_COLUMNS = 24
ACTIONS = {'UP', 'LEFT', 'RIGHT', 'DOWN'}

# Custom map data
SHADE = {(6, 9), (7, 9), (8, 9), (9, 9), (10, 9), (14, 9), (10, 10),
         (11, 10), (12, 10), (13, 10), (10, 12), (11, 12), (10, 13), (11, 13)}

class Maze(Problem):
    
    def __init__(self, initial=None, goal=None):
        Problem.__init__(self, initial=initial, goal=goal)


    def h(self, node):
        return self.h_manhattan(node)

    def h_manhattan(self, node):
        return abs(node.state[0] - self.goal[0]) + abs(node.state[1] - self.goal[1])
    
    def h_geo(self, node):
        return math.sqrt((node.state[0] - self.goal[0])**2 + (node.state[1] - self.goal[1])**2)

    def action_cost(self, s, a, s1): 
        if a in {'LEFT', 'RIGHT'}:
            return 10
        else:
            return 1
                  
    def result(self, state, action):        
        x, y = state
        if action == 'UP':
            return (x + 1, y) if x < MAZE_ROWS - 1 else (x, y)
        if action == 'DOWN':
            return (x - 1, y) if x > 0 else (x, y)
        if action == 'LEFT':
            return (x, y - 1) if y > 0 else (x, y)
        if action == 'RIGHT':
            return (x, y + 1) if y < MAZE_COLUMNS - 1 else (x, y)

    def actions(self, state):       
        x, y = state
        possible_actions = set(ACTIONS)

        if x == 0:
            possible_actions.remove('DOWN')
        if x == MAZE_ROWS - 1:
            possible_actions.remove('UP')
        if y == 0:
            possible_actions.remove('LEFT')
        if y == MAZE_COLUMNS - 1:
            possible_actions.remove('RIGHT')

        # check SHADE
        if 'UP' in possible_actions and (x + 1, y) in SHADE:
            possible_actions.remove('UP')
        if 'DOWN' in possible_actions and (x - 1, y) in SHADE:
            possible_actions.remove('DOWN')
        if 'LEFT' in possible_actions and (x, y - 1) in SHADE:
            possible_actions.remove('LEFT')
        if 'RIGHT' in possible_actions and (x, y + 1) in SHADE:
            possible_actions.remove('RIGHT')

        return possible_actions
    

### Testing the model

In [6]:
#Testing Maze class
#Note that we perform additional tests to validate the modeling

#An instance of Maze problem; initialize the Maze class with appropriate values
# m1 = Maze(initial = None, goal = None)
# Custom map data
m1 = Maze(initial=(8, 10), goal=(11, 9))

# Testing the methods in Maze class
assert m1.result((4,12),'UP') == (5,12)
assert m1.action_cost((4,12),'UP',(5,12)) == 1
assert m1.actions((4,12)) == {'UP', 'LEFT', 'RIGHT', 'DOWN'}

## **Q1.b: Analysis of the Algorithms** [5 Marks]

1. Solve the above Maze problem using the following algorithms

    a. Breadth-First Search

    b. Depth-First Search with Cycle-Check

    c. Iterative-Deepening Search with Cycle-Check

    d. Uniform-Cost Search

    e. A* Search 

    f. Greedy Best-first Search

In [7]:
# breadth-first search
def breadth_first_search(problem):
    frontier = FIFOQueue([Node(problem.initial)])
    reached = {problem.initial}
    max_frontier = 1

    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node, max_frontier
        for child in expand(problem, node):
            s = child.state
            if s not in reached:
                reached.add(s)
                frontier.appendleft(child)

        max_frontier = max(max_frontier, len(frontier))

    return failure, max_frontier

# Testing breadth-first search
p_bfs = Maze(initial=(8, 10), goal=(11, 9))
result, max_frontier = breadth_first_search(p_bfs)
assert result != failure
print(path_states(result))
print(f'Number of expanded nodes: {len(path_states(result)) - 1}\nMax frontier size: {max_frontier}\nPath cost: {result.path_cost}')

[(8, 10), (7, 10), (6, 10), (5, 10), (5, 9), (5, 8), (6, 8), (7, 8), (8, 8), (9, 8), (10, 8), (11, 8), (11, 9)]
Number of expanded nodes: 12
Max frontier size: 34
Path cost: 39


In [8]:
# depth-first search with cycle-check
def depth_first_search(problem):
    frontier = FIFOQueue([Node(problem.initial)])
    reached = {problem.initial}
    max_frontier = 1

    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node, max_frontier
        for child in expand(problem, node):
            s = child.state
            if s not in reached:
                reached.add(s)
                frontier.append(child)

        max_frontier = max(max_frontier, len(frontier))

    return failure, max_frontier


# Testing depth-first search
p_dfs = Maze(initial=(8, 10), goal=(11, 9))
result, max_frontier = depth_first_search(p_dfs)
assert result != failure
print(path_states(result))
print(f'Number of expanded nodes: {len(path_states(
    result)) - 1}\nMax frontier size: {max_frontier}\nPath cost: {result.path_cost}')

[(8, 10), (7, 10), (6, 10), (5, 10), (4, 10), (3, 10), (2, 10), (1, 10), (0, 10), (0, 9), (0, 8), (0, 7), (0, 6), (0, 5), (0, 4), (0, 3), (0, 2), (0, 1), (0, 0), (1, 0), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (3, 8), (4, 8), (4, 7), (4, 6), (4, 5), (4, 4), (4, 3), (4, 2), (4, 1), (4, 0), (5, 0), (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 6), (6, 7), (6, 8), (7, 8), (8, 8), (8, 7), (8, 6), (8, 5), (8, 4), (8, 3), (8, 2), (8, 1), (8, 0), (9, 0), (10, 0), (10, 1), (10, 2), (10, 3), (10, 4), (10, 5), (10, 6), (10, 7), (10, 8), (11, 8), (11, 9)]
Number of expanded nodes: 70
Max frontier size: 65
Path cost: 529


In [9]:
# iterative Deepening search with cycle-check
def depth_limited_search(problem, limit):
    reached = {problem.initial}
    frontier = [(Node(problem.initial), 0, reached)]
    max_frontier = 1
    cutoff_occurred = False

    while frontier:
        node, depth, reached = frontier.pop()

        if problem.is_goal(node.state):
            return node, max_frontier

        if depth < limit:
            for child in expand(problem, node):
                s = child.state
                if s not in reached:
                    frontier.append((child, depth + 1, reached | {s}))

            max_frontier = max(max_frontier, len(frontier))

        elif depth == limit:
            cutoff_occurred = True

    return (cutoff if cutoff_occurred else failure), max_frontier


def iterative_deepening_search(problem):
    for limit in range(1, 256):
        result, max_frontier = depth_limited_search(problem, limit)
        if result != cutoff:
            return result, max_frontier
    return failure, 0


# Testing iterative deepening search
p_ids = Maze(initial=(8, 10), goal=(11, 9))
result, max_frontier = iterative_deepening_search(p_ids)
assert result != failure
print(path_states(result))
print(f'Number of expanded nodes: {len(path_states(
    result)) - 1}\nMax frontier size: {max_frontier}\nPath cost: {result.path_cost}')

[(8, 10), (7, 10), (6, 10), (5, 10), (5, 9), (5, 8), (6, 8), (7, 8), (8, 8), (9, 8), (10, 8), (11, 8), (11, 9)]
Number of expanded nodes: 12
Max frontier size: 19
Path cost: 39


In [10]:
# uniform-cost search
def uniform_cost_search(problem, pq_key=None):
    pq_key = pq_key or (lambda node: node.path_cost)
    frontier = PriorityQueue([Node(problem.initial)], key=pq_key)
    reached = {problem.initial: 0}
    max_frontier = 1

    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node, max_frontier
        for child in expand(problem, node):
            s = child.state
            if s not in reached or child.path_cost < reached[s]:
                reached[s] = child.path_cost
                frontier.add(child)

        max_frontier = max(max_frontier, len(frontier))

    return failure, max_frontier


# Testing uniform-cost search
p_ucs = Maze(initial=(8, 10), goal=(11, 9))
result, max_frontier = uniform_cost_search(p_ucs)
assert result != failure
print(path_states(result))
print(f'Number of expanded nodes: {len(path_states(
    result)) - 1}\nMax frontier size: {max_frontier}\nPath cost: {result.path_cost}')

[(8, 10), (7, 10), (6, 10), (5, 10), (5, 9), (5, 8), (6, 8), (7, 8), (8, 8), (9, 8), (10, 8), (11, 8), (11, 9)]
Number of expanded nodes: 12
Max frontier size: 34
Path cost: 39


In [11]:
# A* search
def a_star_search(problem):
    return uniform_cost_search(problem, pq_key=lambda node: node.path_cost + problem.h(node))


# Testing A* search
p_astar = Maze(initial=(8, 10), goal=(11, 9))

# h1 (Manhattan distance heuristic)
result1, max_frontier1 = a_star_search(p_astar)
assert result1 != failure
print("Result of A* search with Manhattan distance heuristic:")
print(path_states(result1))
print(f'Number of expanded nodes: {len(path_states(
    result1)) - 1}\nMax frontier size: {max_frontier1}\nPath cost: {result1.path_cost}')

# h2 (Geometric distance heuristic)
p_astar.h = p_astar.h_geo
result2, max_frontier2 = a_star_search(p_astar)
assert result2 != failure
print("Result of A* search with Geometric distance heuristic:")
print(path_states(result2))
print(f'Number of expanded nodes: {len(path_states(
    result2)) - 1}\nMax frontier size: {max_frontier2}\nPath cost: {result2.path_cost}')

Result of A* search with Manhattan distance heuristic:
[(8, 10), (7, 10), (6, 10), (5, 10), (5, 9), (5, 8), (6, 8), (7, 8), (8, 8), (9, 8), (10, 8), (11, 8), (11, 9)]
Number of expanded nodes: 12
Max frontier size: 34
Path cost: 39
Result of A* search with Geometric distance heuristic:
[(8, 10), (7, 10), (6, 10), (5, 10), (5, 9), (5, 8), (6, 8), (7, 8), (8, 8), (9, 8), (10, 8), (11, 8), (11, 9)]
Number of expanded nodes: 12
Max frontier size: 34
Path cost: 39


In [12]:
# greedy-best first search
def greedy_best_first_search(problem):
    return uniform_cost_search(problem, pq_key=lambda node: problem.h(node))


# Testing greedy-best first search
p_gbfs = Maze(initial=(8, 10), goal=(11, 9))

# h1 (Manhattan distance heuristic)
result1, max_frontier1 = greedy_best_first_search(p_gbfs)
assert result1 != failure
print("Result of Greedy-best first search with Manhattan distance heuristic:")
print(path_states(result1))
print(f'Number of expanded nodes: {len(path_states(
    result1)) - 1}\nMax frontier size: {max_frontier1}\nPath cost: {result1.path_cost}')

# h2 (Geometric distance heuristic)
p_gbfs.h = p_gbfs.h_geo
result2, max_frontier2 = greedy_best_first_search(p_gbfs)
assert result2 != failure
print("Result of Greedy-best first search with Geometric distance heuristic:")
print(path_states(result2))
print(f'Number of expanded nodes: {len(path_states(
    result2)) - 1}\nMax frontier size: {max_frontier2}\nPath cost: {result2.path_cost}')

Result of Greedy-best first search with Manhattan distance heuristic:
[(8, 10), (9, 10), (9, 11), (10, 11), (11, 11), (12, 11), (13, 11), (14, 11), (14, 10), (15, 10), (15, 9), (15, 8), (14, 8), (13, 8), (12, 8), (11, 8), (11, 9)]
Number of expanded nodes: 16
Max frontier size: 17
Path cost: 61
Result of Greedy-best first search with Geometric distance heuristic:
[(8, 10), (9, 10), (9, 11), (10, 11), (11, 11), (12, 11), (13, 11), (14, 11), (14, 10), (15, 10), (15, 9), (15, 8), (14, 8), (13, 8), (12, 8), (11, 8), (11, 9)]
Number of expanded nodes: 16
Max frontier size: 17
Path cost: 61


2. Identify the number of number of expanded nodes, maximum frontier size, and path-cost for the above algorithms.  Summarize the statistics in the following table.


 Algorithm             |Number of expanded nodes  | Maximum Frontier Size  |  Path Cost                  |
|:---------------------|:-------------------------|:-----------------------|:----------
| Breadth-First Search | 12 | 34 | 39
| DFS with cycle check | 120 | 169 | 939
| IDS with cycle check | 12 | 19 | 39
| UCS                  | 12 | 34 | 39
| A* Search            | 12 | 34 | 39
| GBFS                 | 16 | 17 | 61

3. Compare the performance of informed search algorithms with proposed heuristics. Identify the best performing heuristic and explain.

